In [1]:
import aisdb
from aisdb.database.dbconn import SQLiteDBConn
from aisdb import DBConn, DBQuery, DomainFromPoints
import os
import datetime as dt
#fjern datetime spesifikke imports og endre koden
from datetime import datetime, timedelta
import nest_asyncio
nest_asyncio.apply()

In [2]:
os.chdir('/home/linex/')
os.getcwd()

'/home/linex'

Gule spor for bedre visualisering

In [3]:

def color_tracks(tracks):
    for track in tracks:
        track['color'] = 'yellow'
        yield track

En titt på hele datasettet med 'rensede' spor. Mal for rensing på https://aisviz.gitbook.io/documentation/tutorials/data-cleaning

In [4]:
dbpath='01_01_23_AIS.db' #Samme dbpath i hver celle

In [ ]:
# Set the start and end times for the query
start_time = datetime.strptime("2023-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')
end_time = datetime.strptime("2023-01-02 00:00:00", '%Y-%m-%d %H:%M:%S')

maxdelta = timedelta(hours=24)  # the maximum time interval
distance_threshold = 20000      # the maximum allowed distance (meters) between consecutive AIS messages
speed_threshold = 50            # the maximum allowed vessel speed in consecutive AIS messages
minscore = 1e-6                 # the minimum score threshold for track segment validation

with aisdb.SQLiteDBConn(dbpath=dbpath) as dbconn:
    qry = aisdb.DBQuery(
        dbconn=dbconn, start=start_time, end=end_time,
        callback=aisdb.database.sqlfcn_callbacks.in_time_bbox_validmmsi
    )
    rowgen = qry.gen_qry()
    tracks = aisdb.track_gen.TrackGen(rowgen, decimate=False)
    
    # Split the tracks into segments based on the maximum time interval
    track_segments = aisdb.split_timedelta(tracks, maxdelta)
    
    # Encode the track segments to clean and validate the track data
    tracks_encoded = aisdb.encode_greatcircledistance(track_segments, 
                                                      distance_threshold=distance_threshold, 
                                                      speed_threshold=speed_threshold, 
                                                      minscore=minscore)
    tracks_colored = color_tracks(tracks_encoded)
    
    aisdb.web_interface.visualize(
        tracks_colored,
        open_browser=True,
    )

Definerer domenet med mest aktivitet i henhold til varmekartet

In [5]:
from shapely.geometry import Polygon
from aisdb.gis import Domain

lon_min, lat_min = -96.0, 26.0
lon_max, lat_max = -88.0, 30.5

rect = Polygon([
    (lon_min, lat_min),
    (lon_min, lat_max),
    (lon_max, lat_max),
    (lon_max, lat_min),
    (lon_min, lat_min),
])

domain = Domain(
    name="gulf_rect",
    zones=[{"name": "rect1", "geometry": rect}],
)

# Set the start and end times for the query
start_time = datetime.strptime("2023-01-01 08:00:00", '%Y-%m-%d %H:%M:%S')
end_time = datetime.strptime("2023-01-01 10:00:00", '%Y-%m-%d %H:%M:%S')

Definerer en ny fargefunksjon for å skille MMSI'ene individuelt

In [6]:
import random

def color_tracks2(tracks):
    colors = {}
    for track in tracks:
        mmsi = track.get('mmsi')
        if mmsi not in colors:
            colors[mmsi] = "#{:06x}".format(random.randint(0, 0xFFFFFF))
            track['color'] = colors[mmsi]
        yield track

Overblikk av dataen i løpet av disse to timene på domenet

In [34]:
with aisdb.SQLiteDBConn(dbpath=dbpath) as dbconn:
    qry = aisdb.DBQuery(
        dbconn=dbconn, start=start_time, end=end_time,
        xmin=domain.boundary['xmin'], xmax=domain.boundary['xmax'],
        ymin=domain.boundary['ymin'], ymax=domain.boundary['ymax'],
        callback=aisdb.database.sqlfcn_callbacks.in_time_bbox_validmmsi,
    )
    rowgen = qry.gen_qry()
    
    tracks = aisdb.track_gen.TrackGen(rowgen, decimate=False)
    colored_tracks = color_tracks2(tracks)

    # Visualization
    aisdb.web_interface.visualize(
        colored_tracks,
        domain=domain,
        open_browser=True,
    )

Serving HTTP assets on localhost:3000


Querying database...	done query
Opening a new browser window to display track data. Press Ctrl-C to stop the server and close the webpage


gio: http://localhost:3000/index.html?python=1&z=2: Operation not supported


127.0.0.1:55254 - received: {'msgtype': 'meta'}
127.0.0.1:55256 - received: {'msgtype': 'validrange'}
127.0.0.1:55256 - received: {'msgtype': 'zones'}
Received KeyboardInterrupt, stopping server...


In [7]:
import pandas as pd
import numpy as np

CSV = 'AIS_midlertidig.csv'

In [8]:
#Må holde store filer utenfor wsl miljøet
os.chdir('/mnt/c/users/alexa/downloads')
os.getcwd()

'/mnt/c/users/alexa/downloads'

For videre undersøking lager jeg et nytt datasett med riktig tidsperiode og domene

In [9]:
START_HOUR = 8
END_HOUR   = 10

chunks = pd.read_csv(
    CSV,
    chunksize=100_000,
)

keep = []
for c in chunks:
    t = c["Time"].astype(str)
    hour = t.str.slice(9, 11).astype("int16")
    c = c[(hour >= START_HOUR) & (hour < END_HOUR)]
    if c.empty:
        continue

    lat = pd.to_numeric(c["Latitude"], errors="coerce")
    lon = pd.to_numeric(c["Longitude"], errors="coerce")

    lon = lon.where(lon <= 180, lon - 360)

    mask = (
        lat.between(lat_min, lat_max) &
        lon.between(lon_min, lon_max)
    )

    if mask.any():
        keep.append(c.loc[mask])

df = pd.concat(keep, ignore_index=True)

Med et ryddig datasett kan jeg nå begynne videre analyse